## Final Exam Machine Learning

Davide Tateo; 167275

Francesca Salute; 167284

Nicole Favero; 167340 

Tomás Gonçalves; 167288

In [5]:
#importing required API and modules
import warnings
from tensorflow.keras.utils import img_to_array
from keras.preprocessing.image import array_to_img
from tensorflow.keras.utils import load_img

In [7]:
#loading the image using load_img
org_img= load_img('./images/test/Benign/6299.jpg') 
#Printing original pic
print(org_img)
org_img.show()
# details about the original image is displayed
print("The mode of the original image= ",org_img.mode)
print("The dimensions of the original image= ",org_img.size) 
# converting the given image into an numpy array
img_array = img_to_array(org_img)
print("Details about the image after conversion:") 
#displaying the type of the image 
print("The datatype of the image now = ",type(img_array))
# displaying the datatype of the elements of array
print("The type of the array:", img_array.dtype)
# displaying dimensions of the image
print("The shape of the image is:", img_array.shape)
#reconstructing the image
new_img = array_to_img(img_array)
#Printing reconstructed pic
print(new_img)
new_img.show()

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x21AC06A2510>
The mode of the original image=  RGB
The dimensions of the original image=  (224, 224)
Details about the image after conversion:
The datatype of the image now =  <class 'numpy.ndarray'>
The type of the array: float32
The shape of the image is: (224, 224, 3)
<PIL.Image.Image image mode=RGB size=224x224 at 0x21AC06A1650>
